In [1]:
# Basic imports
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
# Get Player Data for 2023 season.
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
s = Service("/Users/pirani/Desktop/chromedriver")
driver = webdriver.Chrome(service=s, options=chrome_options)
stats_url = f"https://www.basketball-reference.com/leagues/NBA_2023_per_game.html"
driver.get(stats_url)
driver.execute_script("window.scrollTo(1,500)")
time.sleep(1)
toggle_button = driver.find_element(By.ID, "per_game_stats_toggle_partial_table")
toggle_button.click()
with open (f"2023_Predict/players_stats_2023.html", "w") as file:
    file.write(driver.page_source)
        
with open (f"2023_Predict/players_stats_2023.html") as file:
        data = file.read()
soup = BeautifulSoup(data, 'html.parser')
soup.find('tr', class_="thead").decompose()
table = soup.find_all(id="per_game_stats")[0]
player_data = pd.read_html(str(table))[0]
player_data["Year"] = 2023
player_data.to_csv("2023_Predict/player_stats_2023.csv")

In [5]:
teams_url = f"https://www.basketball-reference.com/leagues/NBA_2023_standings.html"
response = requests.get(teams_url)
with open (f"2023_Predict/teams_list_2023.html", "w") as file:
        file.write(response.text)
    
teams_stats_list = []
with open (f"2023_Predict/teams_list_2023.html") as file:
    data = file.read()
soup = BeautifulSoup(data, 'html.parser')
soup.find('tr', class_="thead").decompose()
eastern_table = soup.find_all(id="divs_standings_E")[0]
eastern_data = pd.read_html(str(eastern_table))[0]
eastern_data["Year"] = 2023
eastern_data["Team"] = eastern_data["Eastern Conference"]
    
western_table = soup.find_all(id="divs_standings_W")[0]
western_data = pd.read_html(str(western_table))[0]
western_data["Year"] = 2023
western_data["Team"] = western_data["Western Conference"]
    
del western_data["Western Conference"]
del eastern_data["Eastern Conference"]
teams_stats_list.append(eastern_data)
teams_stats_list.append(western_data)
team_stats = pd.concat(teams_stats_list)    
team_stats.to_csv("2023_Predict/team_2023.csv")

In [6]:
players = pd.read_csv("2023_Predict/player_stats_2023.csv")
del players["Rk"]
del players["Unnamed: 0"]
players["Player"] = players["Player"].str.replace("*","", regex=False)

teams = pd.read_csv("2023_Predict/team_2023.csv")
teams = teams[~teams.Team.str.contains("Division")]
#teams["Team"] = teams["Team"].str.replace("*", "", regex=False)
del teams["Unnamed: 0"]
teams["Team"] = teams["Team"].str.split('(').str[0]
teams["Team"] = teams["Team"].str.replace(u'\xa0', u'')

team_map = {}
with open("abbreviation.txt") as file:
    data = file.readlines()
    for i in data:
        i.replace("\n", "")
        abbreviated, team = i.replace("\n", "").split(",")
        team_map[team] = abbreviated
teams.replace({"Team": team_map},inplace=True)
teams.rename(columns= {'Team':'Tm'}, inplace = True)

training_set = players.merge(teams, how="outer", on=["Tm", "Year"])
training_set = training_set.dropna()
training_set = training_set.apply(pd.to_numeric, errors='ignore')
training_set["GB"] = pd.to_numeric(training_set["GB"].str.replace("—","0"))
training_set.fillna(0)
team_stats.to_csv("2023_Predict/team_2023.csv")
training_set.to_csv("2023_Predict/completed_data_2023.csv")
data = pd.read_csv("2023_Predict/completed_data_2023.csv", index_col=0)
data

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,PF,PTS,Year,W,L,W/L%,GB,PS/G,PA/G,SRS
0,Precious Achiuwa,C,23,TOR,20,0,18.2,2.9,7.0,0.407,...,1.6,8.1,2023,20,24,0.455,12.5,112.0,111.7,0.58
1,OG Anunoby,SF,25,TOR,40,40,37.0,6.7,14.4,0.463,...,3.2,17.9,2023,20,24,0.455,12.5,112.0,111.7,0.58
2,Dalano Banton,PG,23,TOR,22,2,10.1,2.0,4.7,0.423,...,1.1,5.2,2023,20,24,0.455,12.5,112.0,111.7,0.58
3,Scottie Barnes,PF,21,TOR,41,40,34.2,6.0,13.2,0.456,...,2.0,15.4,2023,20,24,0.455,12.5,112.0,111.7,0.58
4,Khem Birch,C,30,TOR,20,0,8.1,1.0,1.6,0.594,...,1.2,2.2,2023,20,24,0.455,12.5,112.0,111.7,0.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Kevin Porter Jr.,PG,22,HOU,40,40,34.0,6.7,15.4,0.437,...,2.7,19.2,2023,10,34,0.227,20.5,109.6,117.5,-7.63
500,Alperen Şengün,C,20,HOU,40,37,27.1,5.9,10.4,0.564,...,3.4,14.9,2023,10,34,0.227,20.5,109.6,117.5,-7.63
501,Jabari Smith Jr.,PF,19,HOU,43,43,29.9,4.3,11.0,0.390,...,3.1,12.1,2023,10,34,0.227,20.5,109.6,117.5,-7.63
502,Jae'Sean Tate,SF,27,HOU,8,2,20.1,2.8,6.6,0.415,...,3.4,6.8,2023,10,34,0.227,20.5,109.6,117.5,-7.63


In [7]:
data.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'W', 'L',
       'W/L%', 'GB', 'PS/G', 'PA/G', 'SRS'],
      dtype='object')

In [ ]:
predictors = ["Age", "G", "GS", "MP", "FG", "FGA", 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'W', 'L', 'W/L%','GB', 'PS/G', 'PA/G', 'SRS']
train = data[~(data["Year"] == 2023)]
test = data[data["Year"] == 2023]
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(train[predictors], train["Share"])